# Process Data for Analysis

In [1]:
import pandas as pd
import warnings
import datetime

warnings.filterwarnings('ignore', category=UserWarning, module='openpyxl')
warnings.filterwarnings("ignore", category=UserWarning)

## Job Openings, Total NonFarm

In [2]:
openings_total_raw = pd.read_excel("data/job_nonfarm.xlsx", 
                               skiprows=13, 
                               header=0,
                              )

In [3]:
openings_total_melt = pd.melt(openings_total_raw, id_vars="Year", var_name="Month")

In [4]:
openings_total_melt['date'] = openings_total_melt["Month"] \
                              + "-" \
                              + openings_total_melt["Year"].astype(str)
openings_total_melt['date'] = pd.to_datetime(openings_total_melt['date'])
openings_total_melt = openings_total_melt[["date", "value"]]

In [5]:
openings_total_final = openings_total_melt.sort_values("date").iloc[:131]
openings_total_final['value'] = openings_total_final['value'].astype(int)
openings_total_final = openings_total_final.reset_index(drop=True)

In [6]:
openings_total_final.to_csv("processed/job_openings_total.csv",
                            index=False
)

## Job Openings, Professional & Business Services

In [7]:
prof_total_raw = pd.read_excel("data/job_prof_bus.xlsx",
                               skiprows=13,
                               header=0
                              )

In [8]:
prof_melt = pd.melt(prof_total_raw, id_vars="Year", var_name="Month")
prof_melt['date'] = pd.to_datetime(
    prof_melt["Month"]
    + "-"
    + prof_melt["Year"].astype(str)
)
prof_melt = prof_melt[["date", "value"]]

In [9]:
prof_final = prof_melt.sort_values("date").iloc[:131]
prof_final['value'] = prof_final['value'].astype(int)
prof_final = prof_final.reset_index(drop=True)

In [10]:
prof_final.to_csv("processed/job_openings_prof.csv",
                  index=False
                 )

## Job Gains, Total

In [11]:
total_gains_raw = pd.read_excel("data/total_job_gains.xlsx",
                                skiprows=15,
                                header=0
                               )

In [12]:
total_gains_melt = pd.melt(total_gains_raw, id_vars="Year", var_name="Qtr")
total_gains_melt['Qtr'] = total_gains_melt['Qtr'].str.replace("tr", "")
total_gains_melt['date'] = pd.to_datetime(
    total_gains_melt["Year"].astype(str)
    + "-"
    + total_gains_melt["Qtr"],
)
# Change to end of Quarter
total_gains_melt['date'] = total_gains_melt['date'] + pd.offsets.QuarterEnd(0)
total_gains_melt = total_gains_melt[['date', 'value']]

In [13]:
total_gains_final = total_gains_melt.sort_values("date").reset_index(drop=True)
total_gains_final = total_gains_final.iloc[:41]
total_gains_final['value'] = total_gains_final['value'].astype(int)

In [14]:
total_gains_final.to_csv("processed/gains_total.csv",
                         index=False)

 ## Job Gains, Professional & Business

In [15]:
prof_gains_raw = pd.read_excel("data/job_gains_prof_bus.xlsx",
                              skiprows=15,
                              header=0
                              )

In [16]:
prof_gains_melt = pd.melt(prof_gains_raw, id_vars="Year", var_name="Qtr")
prof_gains_melt["Qtr"] = prof_gains_melt["Qtr"].str.replace("tr", "")
prof_gains_melt["date"] = pd.to_datetime(
    prof_gains_melt["Year"].astype(str)
    + "-"
    + prof_gains_melt["Qtr"]
)
# Change to end of Quarter
prof_gains_melt["date"] = prof_gains_melt["date"] + pd.offsets.QuarterEnd(0)
prof_gains_melt = prof_gains_melt[["date", "value"]]

In [17]:
prof_gains_final = prof_gains_melt.sort_values("date").reset_index(drop=True)
prof_gains_final = prof_gains_final.iloc[:41]
prof_gains_final["value"] = prof_gains_final["value"].astype(int)

In [18]:
prof_gains_final.to_csv("processed/gains_prof.csv",
                        index=False)

## Job Loss, Total
## Job Loss, Professional & Business
## Hires, Total
## Hires, Professional & Business
These follow the same format as Job Gains, Total & Job Gains, Professional & Business

In [19]:
# In Path, Out Path, Rows to Skip
loss_data_in_out = [
    ("data/total_job_loss.xlsx", "processed/loss_total.csv", 15),
    ("data/job_loss_prof_bus.xlsx", "processed/loss_prof.csv", 15),
]

In [20]:
def loss_raw_to_processed(input_output: tuple[str, str, int]):
    _in = input_output[0]
    _out = input_output[1]
    skip = input_output[2]
    # Read in data
    df_raw = pd.read_excel(_in, skiprows=skip, header=0)
    
    # Melt data for date formatting
    df_melt = pd.melt(df_raw, id_vars="Year", var_name="Qtr")
    df_melt["Qtr"] = df_melt["Qtr"].str.replace("tr", "")
    df_melt["date"] = pd.to_datetime(
        df_melt["Year"].astype(str)
        + "-"
        + df_melt["Qtr"]
    )
    df_melt["date"] = df_melt["date"] + pd.offsets.QuarterEnd(0)
    df_melt = df_melt[["date", "value"]]
    
    # Prepare final output with only valid values and sorted
    df_final = df_melt.sort_values("date").reset_index(drop=True)
    df_final = df_final.dropna()
    df_final["value"] = df_final["value"].astype(int)
    
    # Write
    df_final.to_csv(_out, index=False)
    

In [21]:
for data in loss_data_in_out:
    loss_raw_to_processed(data)

In [22]:
# In Path, Out Path, Rows to Skip
hires_data_in_out = [
    ("data/hires_total.xlsx", "processed/hires_total.csv", 13),
    ("data/hires_prof_bus.xlsx", "processed/hires_prof_bus.csv", 13),
    ("data/total_layoff.xlsx", "processed/layoff_total.csv", 13),
    ("data/prof_layoff.xlsx",  "processed/layoff_prof.csv", 13)
]

In [23]:
def hires_raw_to_processed(input_output: tuple[str, str, int]):
    _in = input_output[0]
    _out = input_output[1]
    skip = input_output[2]
    # Read in data
    df_raw = pd.read_excel(_in, skiprows=skip, header=0)
    
    # Melt data for date formatting
    df_melt = pd.melt(df_raw, id_vars="Year", var_name="Month")
    df_melt["date"] = pd.to_datetime(
        df_melt["Month"]
        + "-"
        + df_melt["Year"].astype(str)
    )
    df_melt = df_melt[["date", "value"]]
    
    # Prepare final output with only valid values and sorted
    df_final = df_melt.sort_values("date").reset_index(drop=True)
    df_final = df_final.dropna()
    df_final["value"] = df_final["value"].astype(int)
    
    # Write
    df_final.to_csv(_out, index=False)

In [24]:
for data in hires_data_in_out:
    hires_raw_to_processed(data)

## College Enrollemnt

In [25]:
enroll_raw = pd.read_csv("data/enrollment.csv")
enroll_final = enroll_raw.rename(columns={
    "Fall": "date",
    "Total": "total",
    "Four or more years": "four",
    "At least 2 but less than 4 years": "two_to_four",
    "Less than 2 years (below associate)": "less_than_two",      
})
enroll_final["date"] = pd.to_datetime(enroll_final["date"].astype(str)) \
                       + pd.offsets.MonthBegin(8)
enroll_final = enroll_final.sort_values("date")
enroll_final.to_csv("processed/enrollment.csv", index=False)